In [1]:
import os
import re
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# Verzeichnis mit Backtest-Ergebnissen (Pickle-Dateien)
RESULTS_DIR = 'results'

# Statische Beschreibungen zu den Strategien (Beispiel)
STRATEGY_DESCRIPTIONS = {
    'MACD': 'Moving Average Convergence Divergence beschreibt ...',
    'RSI': 'Relative Strength Index misst ...',
    'BOLL': 'Bollinger Bands erfassen ...'
}


def get_available_data():
    """Scannt RESULTS_DIR nach Dateien im Format 'STRATEGIE_SYMBOL_returns.pkl'"""
    files = [f for f in os.listdir(RESULTS_DIR) if f.endswith('.pkl')]
    data_map = {}
    pattern = re.compile(r"(.+)_(.+)_returns\.pkl")
    for fname in files:
        match = pattern.match(fname)
        if match:
            strat, symbol = match.groups()
            data_map.setdefault(strat, set()).add(symbol)
    return {s: sorted(list(v)) for s, v in data_map.items()}


def load_returns(strategy: str, symbol: str) -> pd.Series:
    """Lädt die Zeitreihe der Tagesrenditen für die gegebene Strategie/Symbol."""
    path = os.path.join(RESULTS_DIR, f"{strategy}_{symbol}_returns.pkl")
    return pd.read_pickle(path)


# Initialisierung Dash-App
data_map = get_available_data()
strategies = sorted(data_map.keys())
symbols = sorted({sym for syms in data_map.values() for sym in syms})
app = Dash(__name__)

# --- Layout mit 3 Tabs ---
app.layout = html.Div([
    html.H1("Trading Dashboard - Subprojekt 1"),
    dcc.Tabs(id='tabs', value='tab-overview', children=[
        dcc.Tab(label='Übersicht & Vergleichsanalyse', value='tab-overview'),
        dcc.Tab(label='Strategiedetails', value='tab-details'),
        dcc.Tab(label='Strategiebeschreibungen', value='tab-descriptions')
    ]),
    html.Div(id='tabs-content', style={'marginTop': '20px'})
])

# Callback für Tab-Inhalt
def render_overview_tab():
    return html.Div([
        html.Label('Symbol auswählen'),
        dcc.Dropdown(id='overview-symbol', options=[{'label': s, 'value': s} for s in symbols], value=symbols[0]),
        html.Br(),
        html.Label('Strategien vergleichen'),
        dcc.Dropdown(id='overview-strategies', options=[{'label': s, 'value': s} for s in strategies], value=[strategies[0]], multi=True),
        dcc.Graph(id='overview-graph')
    ])


def render_details_tab():
    return html.Div([
        html.Div([html.Label('Strategie'),
                  dcc.Dropdown(id='detail-strategy', options=[{'label': s, 'value': s} for s in strategies], value=strategies[0])],
                 style={'width': '45%', 'display': 'inline-block'}),
        html.Div([html.Label('Symbol'),
                  dcc.Dropdown(id='detail-symbol')],
                 style={'width': '45%', 'display': 'inline-block', 'marginLeft': '5%'}),
        dcc.Graph(id='detail-graph'),
        html.Div(id='detail-stats', style={'marginTop': '20px'})
    ])


def render_descriptions_tab():
    items = []
    for strat, desc in STRATEGY_DESCRIPTIONS.items():
        items.append(html.H4(strat))
        items.append(html.P(desc))
    return html.Div(items)


@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def switch_tab(tab):
    if tab == 'tab-overview':
        return render_overview_tab()
    elif tab == 'tab-details':
        return render_details_tab()
    elif tab == 'tab-descriptions':
        return render_descriptions_tab()
    return html.Div()

# Callback Overview: Vergleichsanalyse
@app.callback(
    Output('overview-graph', 'figure'),
    Input('overview-symbol', 'value'),
    Input('overview-strategies', 'value')
)
def update_overview_graph(symbol, selected_strats):
    if not symbol or not selected_strats:
        return {}
    df_all = pd.DataFrame()
    for strat in selected_strats:
        series = load_returns(strat, symbol)
        series.index = pd.to_datetime(series.index)
        df_all[strat] = series.cumsum()  # kumulierte Rendite
    fig = px.line(df_all, labels={'value': 'Kumulierte Rendite', 'index': 'Datum'},
                  title=f'Vergleich kumulierter Renditen für {symbol}')
    return fig

# Callback Details: Symbol-Dropdown initialisieren
@app.callback(
    Output('detail-symbol', 'options'),
    Output('detail-symbol', 'value'),
    Input('detail-strategy', 'value')
)
def update_detail_symbol(s):
    syms = data_map.get(s, [])
    opts = [{'label': x, 'value': x} for x in syms]
    val = syms[0] if syms else None
    return opts, val

# Callback Details: Graph & Statistiken
@app.callback(
    Output('detail-graph', 'figure'),
    Output('detail-stats', 'children'),
    Input('detail-strategy', 'value'),
    Input('detail-symbol', 'value')
)
def update_detail_outputs(strategy, symbol):
    if not strategy or not symbol:
        return {}, 'Keine Daten ausgewählt'
    df = load_returns(strategy, symbol)
    df.index = pd.to_datetime(df.index)
    fig = px.line(df, x=df.index, y=df.values,
                  labels={'x': 'Datum', 'y': 'Tägliche Rendite'},
                  title=f'{strategy} - {symbol}: Tagesrenditen')
    cum = (1+df).cumprod().iloc[-1] - 1
    avg = df.mean()
    mdd = (df.cumsum() - df.cumsum().cummax()).min()
    stats = html.Ul([
        html.Li(f'Kumulierte Rendite: {cum:.2%}'),
        html.Li(f'Durchschn. tägliche Rendite: {avg:.2%}'),
        html.Li(f'Max. Drawdown: {mdd:.2%}')
    ])
    return fig, stats

if __name__ == '__main__':
    app.run_server(debug=True)


FileNotFoundError: [WinError 3] Das System kann den angegebenen Pfad nicht finden: 'results'

In [4]:
# Bsp Dashboard ohne Richtige Daten
import os
import re
import numpy as np
import pandas as pd
from dash import Dash, html, dcc, Input, Output
import plotly.express as px

# Verzeichnis mit Backtest-Ergebnissen (Pickle-Dateien)
RESULTS_DIR = 'results'

# Statische Beschreibungen zu den Strategien (Beispiel)
STRATEGY_DESCRIPTIONS = {
    'MACD': 'Moving Average Convergence Divergence beschreibt...',
    'RSI': 'Relative Strength Index misst...',
    'BOLL': 'Bollinger Bands erfassen...'
}

# Sicherstellen, dass RESULTS_DIR existiert und Beispieldaten vorliegen
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)
# Dummy-Daten generieren, falls keine .pkl-Dateien vorhanden sind
existing = [f for f in os.listdir(RESULTS_DIR) if f.endswith('.pkl')]
if not existing:
    np.random.seed(42)
    symbols = ['AAPL', 'GOOG', 'MSFT']
    dates = pd.date_range(end=pd.Timestamp.today(), periods=252)
    for strat in STRATEGY_DESCRIPTIONS.keys():
        for sym in symbols:
            returns = pd.Series(np.random.normal(0, 0.01, size=len(dates)), index=dates)
            path = os.path.join(RESULTS_DIR, f"{strat}_{sym}_returns.pkl")
            returns.to_pickle(path)


def get_available_data():
    """Scannt RESULTS_DIR nach Dateien im Format 'STRATEGIE_SYMBOL_returns.pkl'"""
    files = [f for f in os.listdir(RESULTS_DIR) if f.endswith('.pkl')]
    data_map = {}
    pattern = re.compile(r"(.+)_(.+)_returns\.pkl")
    for fname in files:
        match = pattern.match(fname)
        if match:
            strat, symbol = match.groups()
            data_map.setdefault(strat, set()).add(symbol)
    return {s: sorted(list(v)) for s, v in data_map.items()}


def load_returns(strategy: str, symbol: str) -> pd.Series:
    """Lädt die Zeitreihe der Tagesrenditen für die gegebene Strategie/Symbol."""
    path = os.path.join(RESULTS_DIR, f"{strategy}_{symbol}_returns.pkl")
    if not os.path.exists(path):
        # Fallback: generiere zufällige Dummy-Serie
        dates = pd.date_range(end=pd.Timestamp.today(), periods=252)
        return pd.Series(np.random.normal(0, 0.01, size=len(dates)), index=dates)
    return pd.read_pickle(path)

# Initialisierung Dash-App
data_map = get_available_data()
strategies = sorted(data_map.keys())
symbols = sorted({sym for syms in data_map.values() for sym in syms})
app = Dash(__name__)

# --- Layout mit 3 Tabs ---
app.layout = html.Div([
    html.H1("Trading Dashboard - Subprojekt 1"),
    dcc.Tabs(id='tabs', value='tab-overview', children=[
        dcc.Tab(label='Übersicht & Vergleichsanalyse', value='tab-overview'),
        dcc.Tab(label='Strategiedetails', value='tab-details'),
        dcc.Tab(label='Strategiebeschreibungen', value='tab-descriptions')
    ]),
    html.Div(id='tabs-content', style={'marginTop': '20px'})
])

# --- Tab-Inhalte ---
def render_overview_tab():
    return html.Div([
        html.Label('Symbol auswählen'),
        dcc.Dropdown(id='overview-symbol', options=[{'label': s, 'value': s} for s in symbols], value=symbols[0]),
        html.Br(),
        html.Label('Strategien vergleichen'),
        dcc.Dropdown(id='overview-strategies', options=[{'label': s, 'value': s} for s in strategies], value=[strategies[0]], multi=True),
        dcc.Graph(id='overview-graph')
    ])

def render_details_tab():
    return html.Div([
        html.Div([html.Label('Strategie'),
                  dcc.Dropdown(id='detail-strategy', options=[{'label': s, 'value': s} for s in strategies], value=strategies[0])],
                 style={'width': '45%', 'display': 'inline-block'}),
        html.Div([html.Label('Symbol'),
                  dcc.Dropdown(id='detail-symbol')],
                 style={'width': '45%', 'display': 'inline-block', 'marginLeft': '5%'}),
        dcc.Graph(id='detail-graph'),
        html.Div(id='detail-stats', style={'marginTop': '20px'})
    ])

def render_descriptions_tab():
    items = []
    for strat, desc in STRATEGY_DESCRIPTIONS.items():
        items.append(html.H4(strat))
        items.append(html.P(desc))
    return html.Div(items)

@app.callback(
    Output('tabs-content', 'children'),
    Input('tabs', 'value')
)
def switch_tab(tab):
    if tab == 'tab-overview':
        return render_overview_tab()
    elif tab == 'tab-details':
        return render_details_tab()
    elif tab == 'tab-descriptions':
        return render_descriptions_tab()
    return html.Div()

# Callback Overview: Vergleichsanalyse
@app.callback(
    Output('overview-graph', 'figure'),
    Input('overview-symbol', 'value'),
    Input('overview-strategies', 'value')
)
def update_overview_graph(symbol, selected_strats):
    if not symbol or not selected_strats:
        return {}
    df_all = pd.DataFrame()
    for strat in selected_strats:
        series = load_returns(strat, symbol)
        series.index = pd.to_datetime(series.index)
        df_all[strat] = series.cumsum()
    fig = px.line(df_all, labels={'value': 'Kumulierte Rendite', 'index': 'Datum'},
                  title=f'Vergleich kumulierter Renditen für {symbol}')
    return fig

# Callback Details: Symbol-Dropdown initialisieren
@app.callback(
    Output('detail-symbol', 'options'),
    Output('detail-symbol', 'value'),
    Input('detail-strategy', 'value')
)
def update_detail_symbol(s):
    syms = data_map.get(s, [])
    opts = [{'label': x, 'value': x} for x in syms]
    val = syms[0] if syms else None
    return opts, val

# Callback Details: Graph & Statistiken
@app.callback(
    Output('detail-graph', 'figure'),
    Output('detail-stats', 'children'),
    Input('detail-strategy', 'value'),
    Input('detail-symbol', 'value')
)
def update_detail_outputs(strategy, symbol):
    if not strategy or not symbol:
        return {}, 'Keine Daten ausgewählt'
    df = load_returns(strategy, symbol)
    df.index = pd.to_datetime(df.index)
    fig = px.line(df, x=df.index, y=df.values,
                  labels={'x': 'Datum', 'y': 'Tägliche Rendite'},
                  title=f'{strategy} - {symbol}: Tagesrenditen')
    cum = (1+df).cumprod().iloc[-1] - 1
    avg = df.mean()
    mdd = (df.cumsum() - df.cumsum().cummax()).min()
    stats = html.Ul([
        html.Li(f'Kumulierte Rendite: {cum:.2%}'),
        html.Li(f'Durchschn. tägliche Rendite: {avg:.2%}'),
        html.Li(f'Max. Drawdown: {mdd:.2%}')
    ])
    return fig, stats

if __name__ == '__main__':
    app.run(debug=True)



In [5]:
import quantstats as qs
# Beispiel für die Verwendung von QuantStats
qs.reports.html(df, output='report.html')  # Erzeugt einen HTML-Bericht
qs.reports.full(df)  # Zeigt einen vollständigen Bericht im Notebook an   

AttributeError: 'ZMQInteractiveShell' object has no attribute 'magic'